In [1]:
from glob import glob
import os
import sys
import re
import numpy as np
import pandas as pd
import pickle as pkl
import math
import json
import time
import warnings
from tqdm import tqdm
# import matplotlib.pyplot as plt
# %matplotlib inline
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',300)
warnings.filterwarnings('ignore')

In [2]:
## download flight history from adsbexchange and unzip. glob all the json files to search through the data.
flight_filenames = glob('./raw/2020-01-01/2020-01-01-*.json')

In [3]:
def getWuhanOutbound(flights):
    out = pd.DataFrame()
    n = len(flights)
    for file in flights:
        with open(file) as f:
            data = json.load(f)
            flight_data = data['acList']
            for row in tqdm(flight_data):
                if 'From' in row.keys() and row['From'] == 'ZHHH Wuhan Tianhe, China':
                    out = out.append(row, ignore_index=True)
    return out

In [4]:
zhhh = getWuhanOutbound(flight_filenames)

In [12]:
zhhh = pd.read_csv('wuhan_int_airport_outbound_Jan2020.csv')

In [13]:
zhhh.shape

(6686, 58)

In [14]:
zhhh.head(30)

,Alt,AltT,Bad,CMsgs,CNum,Call,CallSus,Cos,Cou,EngMount,EngType,Engines,FSeen,FlightsCount,From,GAlt,HasPic,HasSig,Help,Icao,Id,Interested,Lat,Long,Man,Mdl,Mil,Mlat,Op,OpIcao,PosTime,Rcvr,Reg,ResetTrail,Spd,SpdTyp,Species,Sqk,TSecs,TT,Tisb,To,Trak,TrkH,Trt,Type,Vsi,VsiT,WTC,Year,Stops,Gnd,InHg,Sig,PosStale,TAlt,TTrk,Tag
0,3000.0,0.0,0.0,19.0,63065.0,CSH9362,0.0,"[31.2624, 121.2577, 1577884849652.0, 4000.0, 3...",China,0.0,3.0,2.0,/Date(1577884674266)/,6.0,"ZHHH Wuhan Tianhe, China",3000.0,0.0,0.0,0.0,781374,7869300.0,0.0,31.317600,121.248900,Boeing,Boeing 737NG 800/W,0.0,0.0,Shanghai Airlines,CSH,1.577885e+12,1011157.0,B-1265,1.0,195.0,0.0,1.0,6607.0,273.0,a,0.0,"ZSPD Shanghai Pudong, China",356.0,0.0,2.0,B738,-1408.0,0.0,2.0,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25600.0,0.0,0.0,16.0,43225.0,CSN660,0.0,"[27.63885, 113.42638, 1577884867961.0, 25500.0...",China,0.0,3.0,2.0,/Date(1577884000540)/,10.0,"ZHHH Wuhan Tianhe, China",25600.0,0.0,0.0,0.0,780E06,7867910.0,0.0,27.546480,113.449950,Boeing,Boeing 777 31BER,0.0,0.0,China Southern Airlines,CSN,1.577885e+12,1011157.0,B-2049,1.0,853.0,0.0,1.0,316.0,947.0,a,0.0,"ZGGG Guangzhou Baiyun, China",164.0,0.0,2.0,B773,0.0,0.0,3.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25600.0,0.0,0.0,2.0,43225.0,CSN660,0.0,"[27.63885, 113.42638, 1577884863206.0, None, 2...",China,0.0,3.0,2.0,/Date(1577884863206)/,0.0,"ZHHH Wuhan Tianhe, China",25600.0,0.0,0.0,0.0,780E06,7867910.0,0.0,27.546480,113.449950,Boeing,Boeing 777 31BER,0.0,0.0,China Southern Airlines,CSN,1.577885e+12,1021303.0,B-2049,1.0,853.0,0.0,1.0,316.0,85.0,a,0.0,"ZGGG Guangzhou Baiyun, China",164.0,0.0,2.0,B773,0.0,0.0,3.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,800.0,0.0,0.0,5.0,41473.0,CES2483,0.0,"[22.74843, 113.73332, 1577884832536.0, 2000.0,...",China,0.0,3.0,2.0,/Date(1577884772675)/,0.0,"ZHHH Wuhan Tianhe, China",800.0,0.0,0.0,0.0,780CAC,7867564.0,0.0,22.708790,113.755090,Boeing,Boeing 737NG 89P/W,0.0,0.0,China Eastern Airlines,CES,1.577885e+12,1021303.0,B-1965,1.0,274.0,0.0,1.0,322.0,176.0,a,0.0,"ZJHK Haikou Meilan, China",152.0,0.0,2.0,B738,-832.0,0.0,2.0,2014.0,"[""ZGSZ Shenzhen Bao'an, China""]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3000.0,0.0,0.0,43.0,63065.0,CSH9362,0.0,"[31.2624, 121.2577, 1577884849864.0, 4000.0, 3...",China,0.0,3.0,2.0,/Date(1577883939492)/,0.0,"ZHHH Wuhan Tianhe, China",3000.0,0.0,0.0,0.0,781374,7869300.0,0.0,31.317600,121.248900,Boeing,Boeing 737NG 800/W,0.0,0.0,Shanghai Airlines,CSH,1.577885e+12,1021303.0,B-1265,1.0,195.0,0.0,1.0,6607.0,1009.0,a,0.0,"ZSPD Shanghai Pudong, China",356.0,0.0,2.0,B738,-1408.0,0.0,2.0,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,13200.0,0.0,0.0,2.0,7151.0,CSN6651,0.0,"[31.12953, 114.56609, 1577848923502.0, 3500.0]",China,0.0,3.0,2.0,/Date(1577848712522)/,8.0,"ZHHH Wuhan Tianhe, China",13200.0,0.0,0.0,0.0,780FC6,7868358.0,0.0,31.129530,114.566090,Airbus,Airbus A320 214SL,0.0,0.0,China Southern Airlines,CSN,1.577849e+12,1011157.0,B-8547,1.0,516.0,0.0,1.0,4235.0,233.0,a,0.0,"ZYCC Longjia, Changchun, China",41.0,0.0,2.0,A320,2560.0,0.0,2.0,2016.0,"['ZYTL Zhoushuizi, Dalian, China']",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,34100.0,0.0,0.0,14.0,35371.0,CSN6611,0.0,"[34.01316, 111.37121, 1577848863883.0, 34000.0...",China,0.0,3.0,2.0,/Date(1577848473103)/,6.0,"ZHHH Wuhan Tianhe, China",34100.0,0.0,0.0,0.0,7802F4,7865076.0,0.0,34.002550,111.258490,Boeing,Boeing 737NG 81B/W,0.0,0.0,China Southern Airlines,CSN,1.577849e+12,1011157.0,B-5195,1.0,629.0,0.0,1.0,6267.0,472.0,a,0.0,"ZYCC Longjia, Changchun, China",264.0,0.0,2.0,B738,0.0,0.0,2.0,2007.0,"['ZBTS Tangshan Sannühe, Tangshan,Hebei Provin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,31075.0,0.0,0.0,51.0,5053.0,CES2360,0.0,"[28.08321, 104.054, 1577848863915.0, 31000.0, ...",China,0.0,3.0,2.0,/Date(1577847304152)/,6.0,"ZHHH Wuhan Tianhe, China",31075.0,0.0,0.0,0.0,780856,7866454.0,0.0,28.218150,104.107580,Airbus,Airbus A320 232,0.0,0.0,China Eastern Airlines,CES,1.577849e+12,1011157.0,B-6875,1.0,929.0,0.0,1.0,3515.0,1641.0,a,0.0,"ZSPD Shanghai Pudong, China",19

In [15]:
zhhh_bk = zhhh.copy()

In [63]:
zhhh.to_csv("wuhan_int_airport_outbound_Jan2020.csv", index=False)